![header](header.jpg)

# HANDS ON SESSION FOR HIGH FREQUENCY RADAR DATA PROCESSING

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Run" and select "Run All Above Selected Cell".<br>
***

# Table of Contents
* [1. Introduction](#chapter1)
* [2. Setup](#chapter2)
    * [2.1. Python packages](#section_2_1)
    * [2.2. HFR network and station information](#section_2_2)
    * [2.3. HFR network specification](#section_2_3)
* [3. Inspect total current field dataset](#chapter3)
    * [3.1. Set up a cluster for enabling parallelized access to data](#section_3_1)
    * [3.2. Open dataset from the EU HFR NODE THREDDS Data Server](#section_3_2)
    * [3.3. Select the variable to be inspected](#section_3_3)
    * [3.4. Inspect the entire spatial domain for a variable at a specific time step](#section_3_4)
    * [3.5. Inspect the time series for a variable at a specific location for a specified time range.](#section_3_5)
    * [3.6. Compute the average of a variable over the entire spatial domain for a specified time period.](#section_3_6)
* [4. Inspect radial current field datasets](#chapter4)
    * [4.1. Open dataset from the EU HFR NODE THREDDS Data Server](#section_4_1)
    * [4.2. Select the variable to be inspected](#section_4_2)
    * [4.3. Inspect the entire spatial domain for a variable at a specific time step](#section_4_3)
    * [4.4. Inspect the time series for a variable at a specific location for a specified time range.](#section_4_4)
    * [4.5. Compute the average of a variable over the entire spatial domain for a specified time period.](#section_4_5)
    * [4.6. Stop the cluster.](#section_4_6)

***

## 1. Introduction <a class="anchor" id="chapter1"></a>
[Go back to the "Table of Contents"](#Table-of-Contents)

This notebook focuses on providing basic routines for inspecting the HFR-derived surface current dataset produced by the operational Near Real Time (NRT) workflow of the [European HFR Node](https://www.hfrnode.eu/). The inspection is useful for assessing the datasets and modify the processing and QC parameters, if needed.
The operational routines of the European HFR Node are available on the GitHub repository [https://github.com/LorenzoCorgnati/EU_HFR_NODE_pyHFR](https://github.com/LorenzoCorgnati/EU_HFR_NODE_pyHFR).

The datasets used by this notebook are directly read from the [THREDDS Data Server](https://thredds.hfrnode.eu) of the European HFR Node.

The comprehensive information about HFR networks and stations are read from the operational database of the European HFR Node. In order to insert or modify the information stored in the database, please use the online [webform](https://webform.hfrnode.eu) of the European HFR Node.

<div class="alert alert-block alert-warning">
<b>WARNING</b>

***  
**Don't you have credentials yet to access the online webform of the European HFR Node?** <br>Please go [here](https://webform.hfrnode.eu) to set or recover them.
</div>

## 2. Setup <a class="anchor" id="chapter2"></a>
[Go back to the "Table of Contents"](#Table-of-Contents)

### 2.1. Python packages <a id="section_2_1"></a>

For the notebook to properly run we need to first load the next packages available from the Jupyter Notebook Ecosystem. Please run the `next cell`:

In [1]:
import os
import fsspec
import xarray as xr
import hvplot.xarray
import intake
import cf_xarray
import numpy as np
import panel as pn
from matplotlib import path
import xoak
import pandas as pd
import panel as pn
import math

import warnings
warnings.filterwarnings("ignore")

### 2.2. HFR network and station information <a id="section_2_2"></a>

Now we need to load information and metadata about the HFR networks and their radial stations. Please run the `next cell`:

In [2]:
# Load network information into a dataframe
allNetworkData = pd.read_csv('allNetworkData.csv')
    
# Load station information into a dataframe
allStationData = pd.read_csv('allStationData.csv')

### 2.3. HFR network specification <a id="section_2_3"></a>

Now we need to specify the HFR network whose data are about to be inspected. Please specify the ID of the HFR network you manage and run the `next cell`:

In [3]:
networkID = 'HFR-TirLig' # REPLACE THIS NETWORK WITH THE ID OF THE NETWORK YOU MANAGE

Now we load information and metadata about the specified HFR network and its stations. Please run the `next cell`:

In [4]:
# Network
networkData = allNetworkData.loc[allNetworkData['network_id'] == networkID]

# Stations
stationData = allStationData.loc[allStationData['network_id'] == networkID]

And now we display them. Please run the `next cells`:

In [5]:
# Network
networkData

,network_id,operational_from,operational_to,EDIOS_Series_id,EDMO_code,metadata_page,title,summary,institution_name,citation_statement,...,EU_HFR_processing_flag,connected_to_NRT,connected_to_REP,total_input_folder_path,total_HFRnetCDF_folder_path,total_mat_folder_path,TDS_root_url,SDC_distribution_flag,SDC_OpenDAP_data_url,SDC_folder_path
37,HFR-TirLig,2016-06-22,NaN,HFR-TirLig,134,https://thredds.hfrnode.eu:8443/thredds/NRTcur...,Near Real Time Surface Ocean Velocity by HFR-T...,The data set consists of maps of total velocit...,National Research Council - Institute of Marin...,Data collected and processed by CNR-ISMAR with...,...,1,1.0,1.0,NaN,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Tota...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Tota...,NaN,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...


In [6]:
# Stations
stationData

,station_id,network_id,station_full_name,site_lon,site_lat,operational_from,operational_to,permanent_installation,oceanops_ref,wmo_code,...,radial_QC_radial_count_threshold,number_of_range_cells,EU_HFR_processing_flag,connected_to_NRT,connected_to_REP,radial_input_folder_path,radial_HFRnetCDF_folder_path,SDC_distribution_flag,SDC_OpenDAP_data_url,SDC_folder_path
119,LIGW,HFR-TirLig,Celle Ligure,8.56408,44.3465,2021-12-17,NaN,1,6103569.0,6103569.0,...,200,67,1,1,1,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...
144,MONT,HFR-TirLig,Monterosso al Mare,9.65333,44.1458,2016-06-22,2018-03-26,0,6103578.0,6103578.0,...,200,51,1,0,1,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...
165,PCOR,HFR-TirLig,Punta Corone,9.65930,44.1435,2018-04-04,NaN,1,6103575.0,6103575.0,...,200,51,1,1,1,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...
169,PFIN,HFR-TirLig,Portofino,9.21836,44.2987,2020-07-17,NaN,1,6103570.0,6103570.0,...,200,67,1,1,1,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...
253,TINO,HFR-TirLig,Isola del Tino,9.84922,44.0264,2016-08-04,NaN,1,6103576.0,6103576.0,...,200,51,1,1,1,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...
266,VIAR,HFR-TirLig,Viareggio,10.23730,43.8579,2018-09-13,NaN,1,6103571.0,6103571.0,...,200,51,1,1,1,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/Radi...,1,https://thredds.hfrnode.eu:8443/thredds/dodsC/...,/home/radarcombine/EU_HFR_NODE/HFR-TirLig/SDC/...


## 3. Inspect total current field dataset<a class="anchor" id="chapter3"></a>
[Go back to the "Table of Contents"](#Table-of-Contents)

In this section we'll inspect the total velocity field data produced by your HFR network. In particular, we'll check: **(i) the entire spatial domain for a variable at a specific time step, (ii) the time series of a variable at a specific location for a specified time range, (iii) the average of a variable over the entire spatial domain for a specified time period**. 

### 3.1. Set up a cluster for enabling parallelized access to data<a id="section_3_1"></a>

We'll open the datasets for taking advantage of parallel compute environments using `dask`. We're going to start a **cluster** now so that future steps can take advantage of this ability. 

This is an optional step, but speeds up data loading and processing significantly, especially when accessing data from the cloud. Please run the `next cell`:

In [7]:
from dask.distributed import LocalCluster, Client
cluster = LocalCluster()
client = Client(cluster)

client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 62.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44573,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 62.48 GiB
Comm: tcp://127.0.0.1:39775,Total threads: 4
Dashboard: http://127.0.0.1:46507/status,Memory: 7.81 GiB
Nanny: tcp://127.0.0.1:40803,


### 3.2. Open dataset from the EU HFR NODE THREDDS Data Server<a id="section_3_2"></a>

This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [ ]:
# Get the TDS URL of the dataset
OpenDAPdataUrl = networkData.at[networkData.index.values.astype(int)[0],'SDC_OpenDAP_data_url']

# Read aggregated total dataset from THREDDS catalog via OpenDAP
ds = xr.open_dataset(OpenDAPdataUrl, decode_times=True, chunks={})
ds

### 3.3. Select the variable to be inspected<a id="section_3_3"></a>

Let's select the variable you want to inspect. Please run the `next cell`:

In [ ]:
# Set the variable to be inspected
var = 'EWCT'

# Extract the DataArray for the selected variable
da = ds[var]
da

Use the CF conventions to identify the coordinate variables for longitude, latitude and time. Please run the `next cell`:

In [ ]:
x = da.cf['longitude']
y = da.cf['latitude']
t = da.cf['time']
print(x.name, y.name, t.name)

### 3.4. Inspect the entire spatial domain for a variable at a specific time step<a id="section_3_4"></a>

Select the time step you want to investigate (please use the format ***YYYY-MM-DD hh:mm***). Please run the `next cell`:

In [ ]:
timeStep = '2022-08-01 10:00'

Load the entire spatial domain for the selected variable at the specified time step. This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [ ]:
%time
da2d = da.cf.sel(T=timeStep, method='nearest').load()

Now let's plot the variable content on the entire spatial domain for the selected time step.

We use the `holoviz` package to create the plot: it allows you to **interactively explore the plot**.
By ***hovering*** on it you can display the variable value and the coordinates of the points.
By enabling the ***zoom*** feature (icon of the square with the lens on the right of the colorbar) you can ***shift-drag*** the cursor and select an area to zoom in. You can restore the full view by clicking on the ***reset*** button (two circular arrows).

Don't forget to **set the colorbar range** more suitable for the variable you want to insect by modifying the tuple assigned to ***clim***. Please run the `next cells`:

In [ ]:
# Set the colorbar range
clim = (-1, 1)      # MODIFY THE RANGE ACCORDING TO THE VARIABLE CONTENT

In [ ]:
da2d.hvplot(x=x.name, y=y.name, rasterize=True, geo=True, tiles='OSM', cmap='viridis', clim=clim, title= var + ' variable from ' + networkID + '-Total on ' + timeStep)

### 3.5. Inspect the time series for a variable at a specific location for a specified time range<a id="section_3_5"></a>

Identify the point for which the time series will be created by setting its longitude and latitude coordinates. Please run the `next cell`:

In [ ]:
# Set the point coordinates
lon, lat = 9.8076, 43.9639           # MODIFY THE CCORDINATES FOR CHOOSING THE POINT YOU WANT TO INSPECT

Now set the time range for which you want the time series to be created (please use the format ***YYYY-MM-DD hh:mm***). Please run the `next cell`:

In [ ]:
# Set the start and end time steps for selecting the time range
tStart = '2020-03-01'
tEnd = '2022-10-31'

Before to load the data, let's see what we are selecting. Please run the `next cell`:

In [ ]:
da.cf.sel(X=lon, Y=lat, method="nearest").cf.sel(T=slice(tStart, tEnd))

Load the time series for the selected variable at the specified location and for the specified time range. This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [ ]:
%time
da1d = da.cf.sel(X=lon, Y=lat, method="nearest").cf.sel(T=slice(tStart, tEnd)).load()

Now let's plot the time series of the selected variable for the specified location and time range.

We use the `holoviz` package to create the plot: it allows you to **interactively explore the plot**.
By ***hovering*** on it you can display the variable value and the time step of the points.
By enabling the ***zoom*** feature (icon of the square with the lens on the right of the colorbar) you can ***shift-drag*** the cursor and select an area to zoom in. You can restore the full view by clicking on the ***reset*** button (two circular arrows).

Please run the `next cells`:

In [ ]:
da1d.hvplot(x=t.name, grid=True, title= 'Time series of variable ' + var + ' from ' + networkID + '-Total at location [' + str(lon) + ', ' + str(lat) + ']')

### 3.6. Compute the average of a variable over the entire spatial domain for a specified time period<a id="section_3_6"></a>

Set the time period for which you want the average to be computed (please use the format ***YYYY-MM-DD hh:mm***). Please run the `next cell`:

In [ ]:
# Set the start and end time steps for selecting the time range
tStart = '2020-03-01'
tEnd = '2022-10-31'

Now evaluate the average over the selected time period. This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [ ]:
%time
da_avg = da.cf.sel(T=slice(tStart, tEnd)).mean(dim=t.name).compute()

Now let's plot the variable average on the entire spatial domain for the selected time period.

We use the `holoviz` package to create the plot: it allows you to **interactively explore the plot**.
By ***hovering*** on it you can display the variable value and the coordinates of the points.
By enabling the ***zoom*** feature (icon of the square with the lens on the right of the colorbar) you can ***shift-drag*** the cursor and select an area to zoom in. You can restore the full view by clicking on the ***reset*** button (two circular arrows).

Don't forget to **set the colorbar range** more suitable for the variable you want to insect by modifying the tuple assigned to ***clim***. Please run the `next cells`:

In [ ]:
# Set the colorbar range
clim = (-1, 1)      # MODIFY THE RANGE ACCORDING TO THE VARIABLE CONTENT

In [ ]:
da_avg.hvplot(x=x.name, y=y.name, rasterize=True, geo=True, tiles='OSM', cmap='viridis', clim=clim, title= 'Average of variable ' + var + ' from ' + networkID + '-Total from ' + tStart + ' to ' + tEnd)

## 4. Inspect radial current field datasets<a class="anchor" id="chapter4"></a>
[Go back to the "Table of Contents"](#Table-of-Contents)

In this section we'll inspect the radial velocity field data produced by your HFR network. In particular, we'll check: **(i) the entire spatial domain for a variable at a specific time step, (ii) the time series of a variable at a specific location for a specified time range, (iii) the average of a variable over the entire spatial domain for a specified time period**. 

### 4.1. Open dataset from the EU HFR NODE THREDDS Data Server<a id="section_4_1"></a>

Now we open the radial velocity datasets of each radial station of your HFR network This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [8]:
# Create an empty list for containing the radial datasets
Rds = []

# Loop through radial sites
for Rindex, site in stationData.iterrows():
    # Get the TDS URL of the dataset
    OpenDAPdataUrl = site['SDC_OpenDAP_data_url']

    # Read aggregated total dataset from THREDDS catalog via OpenDAP
    Rds.append(xr.open_dataset(OpenDAPdataUrl, decode_times=True, chunks={}))

### 4.2. Select the variable to be inspected<a id="section_4_2"></a>

Let's select the variable you want to inspect. Please run the `next cell`:

In [9]:
# Set the variable to be inspected
var = 'EWCT'

# Create an empty dictionary for containing the DataArrays of the selected variable for each radial datasets
radDA = {}

# Loop through radial sites
for rad in Rds:
    # Extract the DataArrays for the selected variable
    radDA[rad.attrs['platform_code']] = rad[var]

### 4.3. Inspect the entire spatial domain for a variable at a specific time step<a id="section_4_3"></a>

Select the time step you want to investigate (please use the format ***YYYY-MM-DD hh:mm***). Please run the `next cell`:

In [ ]:
timeStep = '2022-08-01 10:00'

Load the entire spatial domain for the selected variable at the specified time step for each radial site. This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [ ]:
%time

# Create an empty dictionary for containing the DataArrays of the selected variable for each radial datasets
radDAts = {}          

# Loop through radial sites
for key,val in radDA.items():
    radDAts[key] = val.cf.sel(T=timeStep, method='nearest').load()

Now let's plot the variable content on the entire spatial domain for the selected time step for each radial site.

We use the `holoviz` package to create the plot: it allows you to **interactively explore the plot**.
By ***hovering*** on it you can display the variable value and the coordinates of the points.
By enabling the ***zoom*** feature (icon of the square with the lens on the right of the colorbar) you can ***shift-drag*** the cursor and select an area to zoom in. You can restore the full view by clicking on the ***reset*** button (two circular arrows).

Don't forget to **set the colorbar range** more suitable for the variable you want to insect by modifying the tuple assigned to ***clim***. Please run the `next cells`:

In [ ]:
# Set the colorbar range
clim = (-1, 1)      # MODIFY THE RANGE ACCORDING TO THE VARIABLE CONTENT

In [ ]:
# Enable Panel extension
pn.extension()

# Select a single time slice from each dataset for plotting
plots = []
for key,val in radDAts.items():
        x = val.cf['longitude']
        y = val.cf['latitude']
        t = val.cf['time']
        plot = val.hvplot.quadmesh(x=x.name, y=y.name, rasterize=True, geo=True, tiles='OSM', cmap='viridis', clim=clim, title= var + ' variable from ' + key + ' on ' + timeStep)
        plots.append(plot)

# Calculate number of rows and columns for a balanced grid
n_plots = len(plots)
ncols = math.ceil(math.sqrt(n_plots))
nrows = math.ceil(n_plots / ncols)

# Arrange the plots in a grid layout
grid = pn.GridBox(*plots, ncols=ncols)

# Serve or display the layout
grid.servable()

### 4.4. Inspect the time series for a variable at a specific location for a specified time range<a id="section_4_4"></a>

Identify the point for which the time series will be created by setting its longitude and latitude coordinates. Please run the `next cell`:

In [10]:
# Set the point coordinates
lon, lat = 9.8076, 43.9639           # MODIFY THE CCORDINATES FOR CHOOSING THE POINT YOU WANT TO INSPECT

Now set the time range for which you want the time series to be created (please use the format ***YYYY-MM-DD hh:mm***). Please run the `next cell`:

In [ ]:
# Set the start and end time steps for selecting the time range
tStart = '2022-03-01'
tEnd = '2022-10-31'

Load the time series for the selected variable at the specified location and for the specified time range for each radial site. This operation could take a while because it reads the whole aggregated dataset stored on the EU HFR NODE THREDDS Data Server. Please run the `next cell`:

In [32]:
%time

# Create an empty dictionary for containing the DataArrays of the selected variable for each radial datasets
radDAltr = {}          

# Loop through radial sites
for key,val in radDA.items():
    tmpDA = val.cf.sel(X=lon, Y=lat, Z=0, method="nearest").cf.sel(T=slice(tStart, tEnd)).load()
    # Keep only non-empty time series
    if not np.isnan(tmpDA.values).all():
        radDAltr[key] = tmpDA


CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 7.87 μs


Now let's plot the time series of the selected variable for the specified location and time range for each radial site. **Only the non-empty time series will be plotted** (a time series could be empty either because no data is present in that location for that time range or the location is outside the radial site coverage).

We use the `holoviz` package to create the plot: it allows you to **interactively explore the plot**.
By ***hovering*** on it you can display the variable value and the time step of the points.
By enabling the ***zoom*** feature (icon of the square with the lens on the right of the colorbar) you can ***shift-drag*** the cursor and select an area to zoom in. You can restore the full view by clicking on the ***reset*** button (two circular arrows).

Please run the `next cells`:

In [34]:
# Enable Panel extension
pn.extension()

# Select a single time slice from each dataset for plotting
plots = []
for key,val in radDAltr.items():
        x = val.cf['longitude']
        y = val.cf['latitude']
        t = val.cf['time']
        plot = val.hvplot(x=t.name, grid=True, title= 'Time series of variable ' + var + ' variable from ' + key + ' at location [' + str(lon) + ', ' + str(lat) + ']')
        plots.append(plot)

# Calculate number of rows and columns for a balanced grid
n_plots = len(plots)
ncols = math.ceil(math.sqrt(n_plots))
nrows = math.ceil(n_plots / ncols)

# Arrange the plots in a grid layout
grid = pn.GridBox(*plots, ncols=ncols)

# Serve or display the layout
grid.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'df19a5d5-e1db-45ee-98a2-5dbe90d32a50': {'version…

### 4.6. Stop the cluster<a id="section_4_6"></a>

In [ ]:
cluster.shutdown()

In [33]:
len(radDAltr)

3

In [ ]:
val.hvplot(x=t.name, grid=True, title= 'Time series of variable ' + var + ' variable from ' + key + ' at location [' + str(lon) + ', ' + str(lat) + ']')

In [30]:
type(tmpDA).values

In [16]:
val

<xarray.DataArray 'EWCT' (TIME: 5863)> Size: 47kB
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
    DEPTH      float32 4B 0.0
    RNGE       float32 4B 43.76
    BEAR       float32 4B 11.0
  * TIME       (TIME) datetime64[ns] 47kB 2022-03-01 ... 2022-10-31T23:00:00
    LONGITUDE  float32 4B 10.34
    LATITUDE   float32 4B 44.24
Attributes:
    valid_min:            -10000
    valid_max:            10000
    standard_name:        surface_eastward_sea_water_velocity
    long_name:            Surface eastward sea water velocity
    units:                m s-1
    sdn_parameter_name:   Eastward velocity of water current in the water body
    sdn_parameter_urn:    SDN:P01::LCEWZZ01
    sdn_uom_name:         Metres per second
    sdn_uom_urn:          SDN:P06::UVAA
    ancillary_variables:  QCflag OWTR_QC MDFL_QC CSPD_QC VART_QC AVRB_QC RDCT_QC

In [31]:
np.isnan(tmpDA.values).all()

False

In [18]:
val.isnull()

<xarray.DataArray 'EWCT' (TIME: 5863)> Size: 6kB
array([ True,  True,  True, ...,  True,  True,  True])
Coordinates:
    DEPTH      float32 4B 0.0
    RNGE       float32 4B 43.76
    BEAR       float32 4B 11.0
  * TIME       (TIME) datetime64[ns] 47kB 2022-03-01 ... 2022-10-31T23:00:00
    LONGITUDE  float32 4B 10.34
    LATITUDE   float32 4B 44.24

In [19]:
val.values

array([nan, nan, nan, ..., nan, nan, nan])